In [1]:
import epidag as dag
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
scr = '''
PCore test {
    al = 1
    be = 1
    prob ~ beta(al, be)
    x ~ binom(n, prob)    
}
'''

bn = dag.bayes_net_from_script(scr)

In [3]:
data = [
    {'id': 1, 'n': 10, 'x': 4},
    {'id': 2, 'n': 20, 'x': 7}
]

dm = dag.as_data_model(bn, data)

AttributeError: module 'epidag' has no attribute 'form_hierarchy'

In [4]:
p = dm.sample_prior()
dm.evaluate_likelihood(p)

-11.495266343193055

In [5]:
fit = dag.fitting.SIR(dm)
fit.fit(1000)
fit.summarise_posterior()

INFO:root:Initialising
INFO:root:Sampling-Importance
INFO:root:Resampling
           al      be         prob
count  1000.0  1000.0  1000.000000
mean      1.0     1.0     0.370308
std       0.0     0.0     0.084493
min       1.0     1.0     0.121943
25%       1.0     1.0     0.306035
50%       1.0     1.0     0.378291
75%       1.0     1.0     0.423103
max       1.0     1.0     0.673788
N       1000.000000
ESS      870.240011
Dbar       7.182361
pD         0.448435
DIC        7.630796
dtype: float64


In [7]:
fit = dag.fitting.MCMC(dm)
fit.fit(1000)
fit.summarise_posterior()

INFO:root:Initialising
INFO:root:Burning in
INFO:root:Gathering posteriori
INFO:root:Completed
           al      be         prob
count  1000.0  1000.0  1000.000000
mean      1.0     1.0     0.375369
std       0.0     0.0     0.080557
min       1.0     1.0     0.137716
25%       1.0     1.0     0.319600
50%       1.0     1.0     0.377315
75%       1.0     1.0     0.432420
max       1.0     1.0     0.620260
N       1000.000000
ESS      887.316120
Dbar       7.096182
pD         0.392936
DIC        7.489118
dtype: float64


In [15]:
fit = dag.fitting.ABC(dm)
fit.fit(100, test_p=0.1)
fit.summarise_posterior()

INFO:root:Initialising
INFO:root:Testing threshold
INFO:root:Fitting
INFO:root:Gathering posteriori
INFO:root:Finished
          al     be        prob
count  100.0  100.0  100.000000
mean     1.0    1.0    0.366832
std      0.0    0.0    0.034689
min      1.0    1.0    0.311369
25%      1.0    1.0    0.334778
50%      1.0    1.0    0.367846
75%      1.0    1.0    0.397980
max      1.0    1.0    0.424331
N       100.000000
ESS      99.575397
Dbar      6.373474
pD        0.004413
DIC       6.377887
dtype: float64


In [9]:
scr = '''
PCore ex {
    sun ~ unif(0, 1)
    b0 = 1
    heat ~ exp(sun)
    x1 ~ norm(0, 1)
    x2 ~ norm(0, 1)
    b1 ~ unif(10, 20)
    mu1 = b0 + b1*x1 + b2*x2
    mu2 = b0 + b1*x1
    y1 ~ norm(mu1, sd)
    y2 ~ norm(mu2, 1)
}

'''

bn = dag.bayes_net_from_script(scr)
hie = {
    'g': ['sun', 'agent'],
    'agent': ['x1', 'x2','mu1', 'mu2']
}
sm = dag.as_simulation_core(bn, hie=hie, random=['x2'])


h = dag.form_hierarchy(bn, hie)

h.print()

g(heat, b2, sd, b1, b0, sun)
-- agent(mu1, y2, x1, mu2, y1, x2)


In [10]:
pc = sm.generate(exo={'sd'})
ag1 = pc.breed('a1', 'agent')

In [19]:
ag1.get_sampler('y1')(b2= 4, sd =3)

-1.5826879957652007

In [13]:
pc.get_sampler('heat')()

0.7308904995335411

In [12]:
ng = dag.form_hierarchy(bn, hie)
dag.analyse_node_type(bn, ng, True)


Group g
Exogenous    : ['b2', 'sd']
Can be fixed: ['b1', 'b0', 'sun', 'heat']
Can be random: ['sun', 'heat']
Can be actors: ['heat']
--Group agent
  Exogenous    : []
  Can be fixed: ['x1', 'mu2', 'x2', 'y2']
  Can be random: ['x1', 'mu2', 'x2', 'y2', 'y1', 'mu1']
  Can be actors: ['y2', 'y1']


{'g': (['b2', 'sd'], ['b1', 'b0'], ['sun'], ['heat'], [], []),
 'agent': ([], [], ['x1', 'mu2', 'x2'], ['y2'], ['y1'], ['mu1'])}